In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1uJQE3Z_gtAKWkE7gU8olSPuoo72nZyFD/view?usp=sharing
!unzip -q competition.zip -d 'data'
%cd data
!unzip -q training.zip -d "training_data"
!unzip -q validation_text.zip -d "validation_data"
# !unzip -q training/training_info.zip -d 'training_data/info'
# !unzip -q training_data/training_text.zip -d 'training_data/text'
# !unzip -q validation_data/validation_text.zip -d 'validation_data/validation_text'

Downloading...
From: https://drive.google.com/uc?id=1uJQE3Z_gtAKWkE7gU8olSPuoo72nZyFD
To: /content/competition.zip
100% 15.4M/15.4M [00:00<00:00, 45.7MB/s]
/content/data


In [2]:
!gdown --fuzzy https://drive.google.com/file/d/14NFaF2XbuData18WFJXXzwHZIzmr5W89/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=14NFaF2XbuData18WFJXXzwHZIzmr5W89
To: /content/data/testing_text.rar
100% 6.25M/6.25M [00:00<00:00, 52.9MB/s]


In [3]:
!gdown --fuzzy https://drive.google.com/file/d/12JzJZEMfRajtsP7Q0k2NP1vJelx4I0P_/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=12JzJZEMfRajtsP7Q0k2NP1vJelx4I0P_
To: /content/data/indices_training_validation.tar.gz
100% 12.7M/12.7M [00:00<00:00, 54.0MB/s]


In [4]:
!tar -xf indices_training_validation.tar.gz

In [5]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 214 kB/s 
     |████████████████████████████████| 4.4 MB 46.0 MB/s 
     |████████████████████████████████| 1.2 MB 58.5 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 596 kB 73.9 MB/s 
     |████████████████████████████████| 6.6 MB 53.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ecff9dd195d1b527e7daf476d75ac0fccea73af0e3796cc443418360bc69b47b
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
%cd ..

/content


In [7]:
#@title Write BertBaseline.py
%%writefile BertBaseline.py 
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 30 21:30:43 2022

@author: jyotm

script for sentence transformer baseline
use python ..\evaluation.py --ans sentenceTransformer_baseline_results.csv --evl training_eval.csv
for local evaluation of training prediction
"""

import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import os
import sys
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from pprint import pprint
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import sentence_transformers
from subprocess import Popen

# nltk.download('punkt')
N =2000
# MODEL = 'paraphrase-albert-small-v2'
MODEL = 'all-MiniLM-L12-v2'
TRAIN = False


def read_info(file_name):
    xml_data = open(file_name,'r', encoding="utf8").read()
    root = ET.XML(xml_data)
    data = []
    for child in root:
        data.append(subchild.text for subchild in child)
    df = pd.DataFrame(data)
    df.columns = list(s.tag for s in root[0])
    return df

def create_dataset(root, training_files):
    dataset = {}
    #create dictionary from files
    print("loading training files.......")
    for file in tqdm(training_files):
        # print(file)
        raw_text = open(os.path.join(root,file),'r', encoding="utf8").read()
        get_sentences_with_ids(dataset,file,raw_text)
    return dataset

def get_sentences_with_ids(dataset_dct, fname,content):
    #create sent id first
    file_id = int(re.search(r"\d+", fname).group(0))
    paragraphs = content.split('\n')
    for i, para in enumerate(paragraphs):
        sentences = sent_tokenize(para)
        for j, sent in enumerate(sentences):
            if len(sent) < 20:
                continue
                # print("alert!!!!")
                # print("##"*30)
            dataset_dct[f'{file_id}_{i}_{j}'] = sent
    return dataset_dct

def create_id(file_id,para_id,sent_id):
    file_id = int(re.search(r"\d+", file_id).group(0))
    return f'{file_id}_{para_id}_{sent_id}'
    
def get_dataset_info(root,training_info):
    #get dataset from list of files
    dataset_info = {}
    cnt_none = 0
    print("loading training info...... ")
    for file in tqdm(training_info):
        df = read_info(os.path.join(root,file))
        #the df will contain file_names in columns if its empty
        if 'file_names' in df.columns:
            #skip that file in training_info
            df = None
            cnt_none += 1
        else:
            df['source_ids'] = df.apply(lambda x: create_id(x.file_ids, x.para_ids,x.sens_ids), axis=1)
            df['target_ids'] = df.apply(lambda x: create_id(x.tag_file_ids, x.tag_para_ids,x.tag_sens_ids), axis=1)
        dataset_info[file.split(".")[0]] = df
    print(f"Total no of non pleg files in dataset is: {(len(training_info)-cnt_none)} and percentage is: {(len(training_info)-cnt_none)/len(training_info)}")
    return dataset_info

def get_vector_encodings(dataset,model):
    vectors = {}
    # for key in tqdm(dataset.keys()):
    #     vectors[key] = model.encode([dataset[key]])
    #more efficient way
    vectors = model.encode(list(dataset.values()),show_progress_bar=True)
    vectors = {i:j for i,j in zip(dataset.keys(),vectors)}
    return vectors

def get_solution_parapharse_mining(model,dataset):
    #iterate over dataset to  find the cosine distances
    rev_subset = {}
    for a,b in dataset.items():
        rev_subset[b] = a
    #use inbuilt function for paraphrase mining
    solution = sentence_transformers.util.paraphrase_mining(model,list(dataset.values()),query_chunk_size=10000,corpus_chunk_size=60000,show_progress_bar=True, top_k = 100)
    return (solution,rev_subset)
    
def get_solution_pairs(sub_dataset,dataset_info):
    pairs = {}
    source_text_ids = sub_dataset.keys()
    for k in dataset_info.keys():
        df = dataset_info[k]
        if df is not None:
            source_ids = df.source_ids.values
            target_ids = df.target_ids.values
            for x,y in zip(source_ids,target_ids):
                if x in source_text_ids:
                    pairs[x] = y
    return pairs
            
        

def get_subset_dataset(training_files,dataset_info,n=100):
    #only get subset of dataset with n files and their correspodning target files
    corresponding_target_files = {}
    subset_dataset = {}
    for file in dataset_info.keys():
        if dataset_info[file] is not None:
            corresponding_target_files[file] = dataset_info[file]['tag_file_ids'].unique()
        else:
            #the file does not contain pleg and hence the info is empty
            continue
    subset_dataset = {}
    root = 'training_text' if TRAIN else 'validation_text'
    for fname in tqdm(training_files[:n]):
        raw_text = open(os.path.join(root,fname),'r', encoding="utf8").read()
        get_sentences_with_ids(subset_dataset,fname,raw_text)
        #check if the file does contain pleg (or hence occurs in corresponding target files keys)
        key = fname.split('.')[0]
        if key in corresponding_target_files.keys():
            target_files = corresponding_target_files[key]
            for ftg in target_files:
                raw_txt_tg = open(os.path.join(root,ftg + ".newtext"),'r', encoding="utf8").read()
                get_sentences_with_ids(subset_dataset,ftg,raw_txt_tg)
    print("subset dataset created successfully.........")
    return subset_dataset

if __name__ == '__main__':
    subroot = '\\training' if TRAIN else '\\validation'
    os.chdir(r'C:\Users\jyotm\Documents\Data Science Competitions\TextParaphraseDetection\paraphrase\CRI-Comp-2022-Text-Paraphrase-Detection-Challenge\competition_data'+subroot)
    if TRAIN:
        fname = 'training_info\gen_doc0.newinfo'
        print(os.path.isfile(fname))
        df = read_info(fname)
        print(df.columns, df.shape)
        for i in df.columns:
            print(df[i])
        # print(df['tag_plag_sens'][0])
        training_files = os.listdir('training_text')
        training_info = os.listdir('training_info')
        df['source_ids'] = df.apply(lambda x: create_id(x.file_ids, x.para_ids,x.sens_ids), axis=1)
        print(df['source_ids'])

    model = SentenceTransformer(MODEL,device='cuda')
    root = 'training_text' if TRAIN else 'validation_text'
    if TRAIN:
        dataset_info = get_dataset_info('training_info',training_info)
        # sub_dataset = get_subset_dataset(training_files, dataset_info, n = N)
        sub_dataset = create_dataset(root,training_files)
    else:
        #for validation, we don't have info files
        val_files = os.listdir('validation_text')
        sub_dataset = create_dataset(root,val_files)

    # embeddings = get_vector_encodings(sub_dataset,model)
    # print(embeddings['0_1_0'])
    
    #get solution for subseet
    soln, rev_ds = get_solution_parapharse_mining(model,sub_dataset)
    
    print("Paraphrase extracted...........")
    # thresholds = [0.9,0.8,0.7,0.6,0.5]
    thresholds = [0.81,0.82,0.83,0.84,0.85,0.86,0.87,0.88,0.89,0.95]
    
    for th in thresholds:
        key_ind_subset = list(sub_dataset.keys())
        soln_pairs = {}
        for (score, id1,id2) in soln:
            if score > th and key_ind_subset[id1] not in soln_pairs.keys():
                soln_pairs[key_ind_subset[id1]] = key_ind_subset[id2]
        
        save_preds = True
        if save_preds:
            val_df = pd.DataFrame(soln_pairs.items())
            val_df.columns = ['id1','id2']
            df_name = f"..\Val_sentenceTransformer_baseline_results_thres_{th}.csv"
            df_name = df_name.replace('Val','train') if TRAIN else df_name
            val_df.to_csv(df_name,index=False)
        
        if TRAIN:
            gt_pairs = get_solution_pairs(sub_dataset, dataset_info)
            cnt = 0
            total = len(gt_pairs)
            for sol in soln_pairs.keys():
                if sol in gt_pairs.keys():
                    if soln_pairs[sol] == gt_pairs[sol]:
                        cnt += 1
            
            print(f"The accuracy of this method on subset is: {cnt/total}")
            
            
    
    # print(dataset)
    print("#"*20)
    #verify the dataset creation by keys from info file
    # ids = ['0_'+str(i)+'_'+str(j) for i,j in zip(list(df.para_ids.values),list(df.sens_ids.values))]
    # print(ids)
    # print([dataset[sen] for sen in ids])
    # print(df.plag_sens.values)
    # print("#"*20)
    # print([dataset[s] for s in dataset.keys() if s[2:3] == '0'])
    # print(dataset.keys())
    # print(dataset['0_1_0'])
    
    
    
'''

Results:
    on training set: (The scores are calculated using evaluation.py script against training_eval.csv )
        subset size | Model | recall | f1 score | precision | other hyper parameter imp change (time to calculate batches embeddings)
        1000 | 'all-MiniLM-L12-v2' | 0.02 | 0.01 | 0.01 | with corpus_chunk_size = 1k and top_k = 2 (something went wrong here, dk)
        
        1000 | 'all-MiniLM-L12-v2' | 0.289 | 0.058 | 0.032 | with corpus_chunk_size = 10k and top_k = 2  and threshold = 0.5 (time: 3:11)
        1000 | 'all-MiniLM-L12-v2' | 0.289 | 0.061 | 0.035 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.5 (Time: 2:53)
        1000 | 'all-MiniLM-L12-v2' | 0.198 | 0.198 | 0.198 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.9 (Time: 2:53)
        1000 | 'all-MiniLM-L12-v2' | 0.261 | 0.144 | 0.185 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.8 (Time: 2:53)
        1000 | 'all-MiniLM-L12-v2' | 0.283 | 0.057 | 0.096 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.7 (Time: 2:53)
        following are the score with above model but with different thresholds (0.81 to 0.89)
        Thres: 0.81 | 0.82 | 0.83 | 0.84 | 0.85 | 0.86 | 0.87 | 0.88 | 0.89
        Recall: 0.255 | 0.251 | 0.247 | 0.241 | 0.237 | 0.230 | 0.223 | 0.214 | 0.206
        Precision: 0.153| 0.162 | 0.171 | 0.179 | 0.187 | 0.193 | 0.197 | 0.198 | 0.198
        F1-score: 0.192| 0.197 | 0.202 | 0.205 | 0.2094 | 0.2097 | 0.2094 | 0.206 | 0.202
        
        Conclusion: the highest f1 score for subset 1000 is from threshold 0.86
                    the highest f1 score for subset 2000 occurs at threshold 0.87 (The scores for 0.86 & 0.87 is 0.2355 vs 0.2363)
                    for the full dataset highest f1score occurs at thres 0.87 with value 0.2348 (towards 0.88)        
                
        1000 | 'all-MiniLM-L12-v2' | 0.289 | 0.062 | 0.034 | with corpus_chunk_size = 20k and top_k = 50 and threshold = 0.5 (Time: 3:00)
        2000 | 'all-MiniLM-L12-v2' | 0.502 | 0.050 | 0.091 | with corpus_chunk_size = 20k and top_k = 100 and threshold = 0.5 (Time: 5:06)
        full dataset | 'all-MiniLM-L12-v2' | 0.653 | 0.059 | 0.109 | with corpus_chunk_size = 20k and top_k = 100 and threshold = 0.5 (Time: 6:45 ) + misc time: 4:30
        
        1000 | 'paraphrase-albert-small-v2' | 0.29 | 0.067 | 0.04 | with corpus_chunk_size = 10k and top_k = 50 and threshold = 0.5 (Time: 6:25)
        
        
'''

Writing BertBaseline.py


In [8]:
import BertBaseline
import importlib
import numpy as np
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.auto import tqdm

In [9]:
importlib.reload(BertBaseline)

root = '/content/data/training_data/training/info'
training_info = os.listdir(root)
dataset_info = BertBaseline.get_dataset_info(root,training_info)

def get_input_examples(dataset):
    sentence_dataset = []
    for key in dataset.keys():
        if dataset[key] is not None:
            for df in dataset[key].itertuples():
                score = np.random.uniform(low=0.93, high=1.0)
                # sent = dataset[key]['plag_sens'].values[0]
                # target = dataset[key]['tag_plag_sens'].values[0]
                sent = df[5]
                target = df[9]
                input_sample = InputExample(texts=[sent,target], label=score)
                sentence_dataset.append(input_sample)
    return sentence_dataset
dataset = get_input_examples(dataset_info)

# training_dataset,validation_dataset = train_test_split(dataset, test_size=0.1,random_state=42)
training_dataset = dataset
validation_dataset = dataset[-100:]

loading training info...... 


100%|██████████| 4000/4000 [00:07<00:00, 530.41it/s]


Total no of non pleg files in dataset is: 2746 and percentage is: 0.6865


In [10]:
x = pd.read_csv('/content/data/training_eval_new.csv')
print(len(training_dataset))
x.describe()

6212


,id1,id2
count,6212,6212
unique,6212,6212
top,1_0_3,36_0_3
freq,1,1


In [11]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

# Read the dataset
# model_name = 'cross-encoder/stsb-roberta-base'
# model_name = 'all-roberta-large-v1'
# model_name = 'all-mpnet-base-v1'
model_name = 'paraphrase-multilingual-mpnet-base-v2'
train_batch_size = 16
num_epochs = 2
model_save_path = 'output/training_paraphrase_finetuning_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name)

# Convert the dataset to a DataLoader ready for training
logging.info("Setting up dataloader.....")

train_dataloader = DataLoader(training_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

# Development set: Measure correlation between cosine score and gold labels
logging.info("Setting up dataloader dev dataset.....")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(validation_dataset, name='para-dev')

# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))



2022-07-01 03:28:24 - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

2022-07-01 03:29:01 - Use pytorch device: cuda
2022-07-01 03:29:01 - Setting up dataloader.....
2022-07-01 03:29:01 - Setting up dataloader dev dataset.....
2022-07-01 03:29:01 - Warmup-steps: 78


In [12]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=200,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/389 [00:00<?, ?it/s]

2022-07-01 03:30:07 - EmbeddingSimilarityEvaluator: Evaluating the model on para-dev dataset in epoch 0 after 200 steps:
2022-07-01 03:30:07 - Cosine-Similarity :	Pearson: -0.0498	Spearman: -0.0402
2022-07-01 03:30:07 - Manhattan-Distance:	Pearson: -0.0487	Spearman: -0.0332
2022-07-01 03:30:07 - Euclidean-Distance:	Pearson: -0.0495	Spearman: -0.0322
2022-07-01 03:30:07 - Dot-Product-Similarity:	Pearson: -0.1809	Spearman: -0.1837
2022-07-01 03:30:07 - Save model to output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-07-01_03-28-24
2022-07-01 03:30:58 - EmbeddingSimilarityEvaluator: Evaluating the model on para-dev dataset after epoch 0:
2022-07-01 03:30:59 - Cosine-Similarity :	Pearson: -0.0691	Spearman: -0.0514
2022-07-01 03:30:59 - Manhattan-Distance:	Pearson: -0.0497	Spearman: -0.0334
2022-07-01 03:30:59 - Euclidean-Distance:	Pearson: -0.0486	Spearman: -0.0302
2022-07-01 03:30:59 - Dot-Product-Similarity:	Pearson: -0.1910	Spearman: -0.1817
2022-0

Iteration:   0%|          | 0/389 [00:00<?, ?it/s]

2022-07-01 03:31:53 - EmbeddingSimilarityEvaluator: Evaluating the model on para-dev dataset in epoch 1 after 200 steps:
2022-07-01 03:31:53 - Cosine-Similarity :	Pearson: -0.0262	Spearman: -0.0107
2022-07-01 03:31:53 - Manhattan-Distance:	Pearson: -0.0115	Spearman: -0.0054
2022-07-01 03:31:53 - Euclidean-Distance:	Pearson: -0.0090	Spearman: 0.0062
2022-07-01 03:31:53 - Dot-Product-Similarity:	Pearson: -0.1216	Spearman: -0.1166
2022-07-01 03:31:53 - Save model to output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-07-01_03-28-24
2022-07-01 03:32:44 - EmbeddingSimilarityEvaluator: Evaluating the model on para-dev dataset after epoch 1:
2022-07-01 03:32:44 - Cosine-Similarity :	Pearson: -0.0067	Spearman: 0.0108
2022-07-01 03:32:44 - Manhattan-Distance:	Pearson: 0.0030	Spearman: 0.0084
2022-07-01 03:32:44 - Euclidean-Distance:	Pearson: 0.0059	Spearman: 0.0187
2022-07-01 03:32:44 - Dot-Product-Similarity:	Pearson: -0.1061	Spearman: -0.0996
2022-07-01 0

In [13]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
val_sentences = {}
for filename in tqdm(os.listdir('/content/data/validation_data/validation_text')):
    ind_filename = filename.replace('newtext','xml')
    doc_number = ind_filename.split('.')[0][7:]
    df = BertBaseline.read_info(f'/content/data/indices/{ind_filename}')
    df['indices'] = df['indices'].apply(pd.eval)
    for index, row in df.iterrows():
        sens_id = "_".join([doc_number, str(row['indices'][0]), str(row['indices'][1])])
        # print(doc_number, filename, row['indices'], sens_id)
        val_sentences[sens_id] = row['sentences']
len(val_sentences)

  0%|          | 0/3000 [00:00<?, ?it/s]

98172

In [ ]:

# importlib.reload(BertBaseline)
# root = '/content/data/validation_data/validation_text'

# val_dataset = BertBaseline.create_dataset(root,os.listdir(root))
# len(val_dataset)

loading training files.......


100%|██████████| 3000/3000 [00:02<00:00, 1078.38it/s]


95845

In [15]:
# model_save_path = 'all-roberta-large-v1'
# model_save_path = '/content/output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-06-29_06-24-19'

model = SentenceTransformer(model_save_path)

sol, _ = BertBaseline.get_solution_parapharse_mining(model,val_sentences)


2022-07-01 03:33:36 - Load pretrained SentenceTransformer: output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-07-01_03-28-24
2022-07-01 03:33:40 - Use pytorch device: cuda


Batches:   0%|          | 0/3068 [00:00<?, ?it/s]

In [16]:
import pandas as pd

thresholds = [0.5, 0.81,0.88,0.94,0.95, 0.96, 0.97, 0.963,0.965,0.967, 0.98, 0.975, 0.99, 0.972,0.973,0.976]
    
# thresholds = [0.965, 0.966, 0.967, 0.968, 0.969, 0.9695, 0.97, 0.9705, 0.971, 0.9715, 0.972, 0.973]
for th in thresholds:
    key_ind_subset = list(val_sentences.keys())
    soln_pairs = {}
    for (score, id1,id2) in sol:
        if 1.0 > score > th and key_ind_subset[id1] not in soln_pairs.keys():
            soln_pairs[key_ind_subset[id1]] = key_ind_subset[id2]
    
    save_preds = True
    if save_preds:
        val_df = pd.DataFrame(soln_pairs.items())
        val_df.columns = ['id1','id2']
        df_name = f"output/Val_results_old_method_thres_{th}.csv"
        val_df.to_csv(df_name,index=False)

In [17]:
res_thresholds = [0.94,0.95, 0.96, 0.97, 0.963,0.965,0.967, 0.98, 0.975, 0.99, 0.972,0.973,0.976]
for th in res_thresholds:
    !echo "predicting for {th}"
    !python /content/evaluation.py --ans '/content/output/Val_results_old_method_thres_{th}.csv' --evl /content/validation_eval_new.csv

predicting for 0.94
100% 15993/15993 [00:11<00:00, 1414.99it/s]
100% 15993/15993 [00:02<00:00, 6527.30it/s]
Recall: 0.9446354883081155
Precision: 0.171762646157694
F1-score: 0.29067245119305857
predicting for 0.95
100% 9560/9560 [00:03<00:00, 2398.05it/s]
100% 9560/9560 [00:01<00:00, 6878.87it/s]
Recall: 0.9167812929848693
Precision: 0.2788702928870293
F1-score: 0.4276547962784729
predicting for 0.96
100% 6321/6321 [00:01<00:00, 3617.63it/s]
100% 6321/6321 [00:00<00:00, 7177.37it/s]
Recall: 0.8562585969738652
Precision: 0.3939250118652112
F1-score: 0.539603423989598
predicting for 0.97
100% 4432/4432 [00:00<00:00, 5167.28it/s]
100% 4432/4432 [00:00<00:00, 7952.63it/s]
Recall: 0.7616918844566712
Precision: 0.49977436823104693
F1-score: 0.6035422343324249
predicting for 0.963
100% 5688/5688 [00:01<00:00, 4000.43it/s]
100% 5688/5688 [00:00<00:00, 7491.53it/s]
Recall: 0.8328748280605227
Precision: 0.4258087201125176
F1-score: 0.5635179153094463
predicting for 0.965
100% 5286/5286 [00:01<00

In [18]:
# thresholds = [0.965, 0.966, 0.967, 0.968, 0.969, 0.9695, 0.97, 0.9705, 0.971, 0.9715, 0.972, 0.973, 0.975, 0.979, 0.981, 0.985, 0.99, 0.995]
thresholds = [0.5, 0.7, 0.81,0.85,0.90, 0.94,0.95, 0.957, 0.96, 0.97, 0.963,0.965,0.967, 0.98, 0.975, 0.99, 0.972,0.973,0.976]
for th in thresholds:
    key_ind_subset = list(val_sentences.keys())
    soln_pairs = {}
    scores_key = {}
    for (score, id1,id2) in sol:
        if 1.0 > score > th:
            if key_ind_subset[id1] not in soln_pairs.keys():
                soln_pairs[key_ind_subset[id1]] = [key_ind_subset[id2]]
                scores_key[key_ind_subset[id1]] = [score]
            else:
                soln_pairs[key_ind_subset[id1]].append(key_ind_subset[id2])
                scores_key[key_ind_subset[id1]].append(score)  
    filtered_pairs = {}
    for k in soln_pairs.keys():
        filtered_pairs[k] = soln_pairs[k][0]

    lst = []
    new_pairs = {}
    for k in filtered_pairs.keys():
        id2 = filtered_pairs[k]
        if id2 in filtered_pairs.keys():
            if 1.0 > scores_key[id2][0] > scores_key[k][0]:
                new_pairs[id2] = filtered_pairs[id2]
                lst.append(id2)
            else:
                new_pairs[k] = id2
        else:
            new_pairs[k] = id2

    val_df = pd.DataFrame(new_pairs.items())
    val_df.columns = ['id1','id2']
    df_name = f"output/filtered_with_mpnet_v2_results_{th}.csv"
    val_df.to_csv(df_name,index=False)

In [19]:
for th in thresholds:
    !echo "predicting for {th}"
    !python /content/evaluation.py --ans '/content/output/filtered_with_mpnet_v2_results_{th}.csv' --evl /content/validation_eval_new.csv

predicting for 0.5
100% 15252/15252 [00:10<00:00, 1444.04it/s]
100% 15252/15252 [00:02<00:00, 6584.36it/s]
Recall: 0.9487620357634112
Precision: 0.18089430894308944
F1-score: 0.3038546255506608
predicting for 0.7
100% 15252/15252 [00:10<00:00, 1496.52it/s]
100% 15252/15252 [00:02<00:00, 6532.63it/s]
Recall: 0.9487620357634112
Precision: 0.18089430894308944
F1-score: 0.3038546255506608
predicting for 0.81
100% 15252/15252 [00:10<00:00, 1482.91it/s]
100% 15252/15252 [00:02<00:00, 6505.26it/s]
Recall: 0.9487620357634112
Precision: 0.18089430894308944
F1-score: 0.3038546255506608
predicting for 0.85
100% 15252/15252 [00:10<00:00, 1402.90it/s]
100% 15252/15252 [00:02<00:00, 6524.69it/s]
Recall: 0.9487620357634112
Precision: 0.18089430894308944
F1-score: 0.3038546255506608
predicting for 0.9
100% 15252/15252 [00:10<00:00, 1495.14it/s]
100% 15252/15252 [00:02<00:00, 6561.71it/s]
Recall: 0.9487620357634112
Precision: 0.18089430894308944
F1-score: 0.3038546255506608
predicting for 0.94
100% 120

#filtering out using cross encoder roberta trained on st benchmark

In [20]:
from sentence_transformers import CrossEncoder
cross_enc = CrossEncoder('cross-encoder/stsb-roberta-large')

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/139 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

2022-07-01 03:40:07 - Use pytorch device: cuda


In [21]:
import pandas as pd

preds = pd.read_csv('/content/output/filtered_with_mpnet_v2_results_0.96.csv')

pairs = []
ids = []
for i,(k,v) in preds.iterrows():
    pairs.append((val_sentences[k], val_sentences[v]))
    ids.append((k,v))

# model_save_path = 'cross-encoder/stsb-roberta-large'
# cross_enc = CrossEncoder(model_save_path)
predictions_confs = cross_enc.predict(pairs)


Batches:   0%|          | 0/175 [00:00<?, ?it/s]

In [22]:
predictions_confs[-20:]

array([0.44375882, 0.7402931 , 0.7123775 , 0.41469103, 0.44733307,
       0.5058074 , 0.6218172 , 0.7619903 , 0.76495427, 0.18672492,
       0.48864603, 0.36046687, 0.7327351 , 0.74158037, 0.8341332 ,
       0.88065964, 0.62419355, 0.6863878 , 0.34046298, 0.3799331 ],
      dtype=float32)

In [23]:
for th in [0.2, 0.5, 0.55, 0.57,  0.59, 0.6, 0.61, 0.62,0.63,0.64, 0.65, 0.66, 0.67,  0.69, 0.7, 0.75, 0.8, 0.85, 0.89, 0.92, 0.95]:
# for th in [0.96, 0.9615, 0.9620, 0.9631, 0.967, 0.968, 0.969, 0.9695]:
    soln_pairs = {}
    for i,p in enumerate(predictions_confs):
        #the id2 filtering is based on similarity scores from first model predictions as the dataframe contained 
        #ordered/sorted pairs with highest similarity first
        if  p > th and not ids[i][1] in soln_pairs.values():
            soln_pairs[ids[i][0]] = ids[i][1]
    
    #check if the id2 is present in ids1 keys and if the similarity score is smaller than that pair, include other pair else add current pair
    #basically, if score(id1,id2) > score(id2,id3) include id1,id2 pair else include id2,id3 pair
    new_pairs = {}
    for k in soln_pairs.keys():
        id2 = soln_pairs[k]
        if id2 in soln_pairs.keys() and predictions_confs[[x[0] for x in ids].index(id2)] > predictions_confs[[x[0] for x in ids].index(k)]:
            new_pairs[id2] = soln_pairs[id2]
        else:
            new_pairs[k] = id2

    print(f"percentage retained for {th} is before filtering: ", len(soln_pairs)/len(ids))
    print(f"percentage retained for {th} is after filtering: ", len(new_pairs)/len(ids))
    val_df = pd.DataFrame(new_pairs.items())
    val_df.columns = ['id1','id2']
    df_name = f"output/cross_enc_new_result_{th}.csv"
    val_df.to_csv(df_name,index=False)
    !python /content/evaluation.py --ans '/content/output/cross_enc_new_result_{th}.csv' --evl /content/validation_eval_new.csv
    # !python /content/evaluation.py --ans '/content/output/cross_enc_result_{th}.csv' --evl /content/validation_eval_new.csv


percentage retained for 0.2 is before filtering:  0.8702412868632707
percentage retained for 0.2 is after filtering:  0.8570151921358355
100% 4795/4795 [00:01<00:00, 4773.76it/s]
100% 4795/4795 [00:00<00:00, 8093.82it/s]
Recall: 0.8449105914718019
Precision: 0.5124087591240876
F1-score: 0.6379332727508762
percentage retained for 0.5 is before filtering:  0.8078641644325291
percentage retained for 0.5 is after filtering:  0.7987488829311885
100% 4469/4469 [00:00<00:00, 5093.31it/s]
100% 4469/4469 [00:00<00:00, 8144.32it/s]
Recall: 0.842503438789546
Precision: 0.5482210785410606
F1-score: 0.6642266503998916
percentage retained for 0.55 is before filtering:  0.7846291331546024
percentage retained for 0.55 is after filtering:  0.7762287756925826
100% 4343/4343 [00:00<00:00, 5375.90it/s]
100% 4343/4343 [00:00<00:00, 8319.39it/s]
Recall: 0.8407840440165062
Precision: 0.5629749021413769
F1-score: 0.6743897393462971
percentage retained for 0.57 is before filtering:  0.7758713136729223
percenta

Prediction Analysis

In [ ]:
val_soln = pd.read_csv('/content/validation_eval_new.csv')
# val_soln[val_soln['id1'] == '4900_1_3']
z = 0
# for x, (id1,id2) in val_soln.iterrows():
    # print(id1, val_dataset[id1])
    # print(id2, val_dataset[id2])
    # print("###"* 10)
    # z += 1
    # if z > 10: 
    #     break

ids1 = val_soln.id1.unique().tolist()
ids2 = val_soln.id2.unique().tolist()
s1 = set(ids1) 
s2 = set(ids2)
print(len(ids1), len(ids2), len(s1), len(s2))
len(s1-set(val_sentences.keys())), s1.intersection(s2), len(s2-set(val_sentences.keys()))

2908 2908 2908 2908


(0, set(), 0)

In [ ]:
z = 0
for x, (id1,id2) in val_soln.iterrows():
    print(id1, val_sentences[id1])
    print(id2, val_sentences[id2])
    print("###"* 10)
    z += 1
    if z > 10: 
        break

4002_1_3 One meta-analysis found weaker evidence for TGF-1 869C/T polymorphism and breast cancer risk, while the other meta-analysis looked at the association between 869C/T polymorphisms and HNC.
5670_5_0 One meta-analysis found weaker evidence for TGF-1 869C/T polymorphism and breast cancer risk, while the other meta-analysis looked at the association between 869C/T and HNC.
##############################
4002_4_1 Figure 3 shows the statistical significance of differentially expressed circRNAs between cases and controls.
6973_0_2 A volcano plot shows the statistical significance of differentially expressed circRNAs between cases and controls.
##############################
4004_1_1 The youngest subject in the study was 4 months old.
5889_2_2 The youngest subject in our study population was 4 months old.
##############################
4004_5_5 Hummel et al.
4572_3_4 Hummel et al. were in 1997.
##############################
4005_0_2 It is possible that high attentional impulsivity is 

In [ ]:
sols = pd.read_csv('/content/output/cross_enc_new_result_0.65.csv')
def reverse(pair):
    return [pair[1],pair[0]]

val_gt = []
for row in val_soln.iterrows():
    val_gt.append([row[1]['id1'],row[1]['id2']])

tp = 0
for x, (id1,id2) in sols.iterrows():
    if not [id1,id2] in val_gt and not [id1, id2] in val_gt:
        print(id1, val_sentences[id1])
        print(id2, val_sentences[id2])
        tp += 1
    else:
        print("present")
    print("----"*10)
    if tp > 10:
        break

4365_3_0 In the presence of bispecific anti-CD3/CD3 figure C, the yields of CD8 SP progeny generated by class I–deficient DPlo cells increased moderately, and they generated CD8 SP as well as CD4.
6203_2_6 In the presence of bispecific anti-CD3/CD3 figure C, the yields of CD8 SP progeny generated by class I–deficient DPlo cells increased moderately, and they generated CD8 SP as well as CD4.
----------------------------------------
5922_2_5 Most of the changes in secretory profile were reversed by the use of cholesterol.
5030_2_1 Most of the changes in secretory profile were reversed by the use of cholesterol.
----------------------------------------
5922_3_4 Only a discrimination of CD16pos and CD16neg NK cells was implemented for further analysis of the maturation state, as all stimulation protocols caused an upregulation of the CD56 cell marker.
5217_3_5 Only a discrimination of CD16pos and CD16neg NK cells was implemented for further analysis of the maturation state, as all stimulat

In [33]:
sols = pd.read_csv('/content/output/cross_enc_new_result_0.66.csv')
new_pairs = {}
duplicate = 0
for x, (id1,id2) in sols.iterrows():
    if val_sentences[id1] != val_sentences[id2]:
        duplicate += 1
        new_pairs[id1] = id2

print(f"percentage retained for is after filtering: ", len(new_pairs)/len(sols))
val_df = pd.DataFrame(new_pairs.items())
val_df.columns = ['id1','id2']
df_name = f"output/cross_enc_new_result_final.csv"
val_df.to_csv(df_name,index=False)
print(f"Length {duplicate}")

percentage retained for is after filtering:  0.8343373493975904
Length 3324


In [34]:
!python /content/evaluation.py --ans '/content/output/cross_enc_new_result_final.csv' --evl /content/validation_eval_new.csv

100% 3324/3324 [00:00<00:00, 6813.84it/s]
100% 3324/3324 [00:00<00:00, 9086.27it/s]
Recall: 0.8187757909215956
Precision: 0.7163056558363418
F1-score: 0.7641206675224648


WOW, Another breakthrough!!, removing duplicate entries gave another huge score boost!!

In [ ]:
sols = pd.read_csv('/content/output/cross_enc_new_result_final.csv')
def reverse(pair):
    return [pair[1],pair[0]]

val_gt = []
for row in val_soln.iterrows():
    val_gt.append([row[1]['id1'],row[1]['id2']])

tp = 0
for x, (id1,id2) in sols.iterrows():
    if not [id1,id2] in val_gt and not [id1, id2] in val_gt:
        print(id1, val_sentences[id1])
        print(id2, val_sentences[id2])
        tp += 1
    else:
        print("present")
    print("----"*10)
    if tp > 10:
        break

6209_2_1 The findings support the idea that Nurr1 exerts a protective effect on adult mDA neurons in a cell-autonomous manner.
5774_3_3 The findings support the idea that Nurr1 exerts a protective effect on adult mDA neuron in a cell-autonomous manner.
----------------------------------------
6337_2_0 HDI and risk description.
6594_3_3 Risk Description and HDI.
----------------------------------------
6362_3_0 Figure 1.
4340_0_0 Figure 1
----------------------------------------
4215_0_0 The coding sequence alone is 15.2 kb in length, and it turned out to be a Herculean task to find the cause of the candidate genes in humans and pigs.
4699_0_5 The coding sequence alone is 15.2 kb in length and it turned out to be a Herculean task to find the cause of the candidate genes in humans and pigs.
----------------------------------------
6491_5_2 The recent initiatives proposed by the National Tuberculosis Control program are encouraging.
5190_3_2 The recent initiatives proposed by the National

In [ ]:
for x,y in zip(val_sentences['6491_5_2'],val_sentences['5190_3_2']):
    if x != y:
        print(x,y)

p P


There still exists some dissimilarities between these sentences, removing them would need some manual post processing but will gives good results

In [ ]:
tp = 0
lower = 0
for x, (id1,id2) in sols.iterrows():
    if len(val_sentences[id1]) == len(val_sentences[id2]):
        tp += 1
        print(id1, val_sentences[id1])
        print(id2, val_sentences[id2])
        print("---"*20)
tp

6337_2_0 HDI and risk description.
6594_3_3 Risk Description and HDI.
------------------------------------------------------------
6491_5_2 The recent initiatives proposed by the National Tuberculosis Control program are encouraging.
5190_3_2 The recent initiatives proposed by the National Tuberculosis Control Program are encouraging.
------------------------------------------------------------
5621_1_3 A single follow-up period closest to the median of the other studies will be chosen for inclusion in the meta-analysis, where studies present effects on the prospective association at multiple follow-up times.
5137_4_0 A single follow-up period closest to the median of the other studies will be chosen for inclusion in the meta-analysis, where studies present effects on the prospective association at multiple follow up times.
------------------------------------------------------------
5759_4_4 The size of the set of known primary metabolic reactions in Streptomyces species would not be 

91

## Test Inference

In [35]:
!gdown --fuzzy https://drive.google.com/file/d/1_J7HMV87UqAem-ogJIkyPazxqunCvL4H/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1_J7HMV87UqAem-ogJIkyPazxqunCvL4H
To: /content/indices_testing.zip
100% 7.73M/7.73M [00:00<00:00, 47.5MB/s]


In [ ]:
!unrar x -r /content/data/testing_text.rar

In [37]:
!unzip -q indices_testing.zip

In [40]:
test_sentences = {}
for filename in tqdm(os.listdir('/content/text')):
    ind_filename = filename.replace('newtext','xml')
    doc_number = ind_filename.split('.')[0][7:]
    df = BertBaseline.read_info(f'/content/indices/{ind_filename}')
    df['indices'] = df['indices'].apply(pd.eval)
    for index, row in df.iterrows():
        sens_id = "_".join([doc_number, str(row['indices'][0]), str(row['indices'][1])])
        # print(doc_number, filename, row['indices'], sens_id)
        test_sentences[sens_id] = row['sentences']
len(test_sentences)

  0%|          | 0/3000 [00:00<?, ?it/s]

97906

In [41]:

test_sol, _ = BertBaseline.get_solution_parapharse_mining(model,test_sentences)


Batches:   0%|          | 0/3060 [00:00<?, ?it/s]

In [42]:
key_ind_subset = list(test_sentences.keys())
soln_pairs = {}
scores_key = {}
for (score, id1,id2) in test_sol:
    if 1.0 > score > 0.96:
        if key_ind_subset[id1] not in soln_pairs.keys():
            soln_pairs[key_ind_subset[id1]] = [key_ind_subset[id2]]
            scores_key[key_ind_subset[id1]] = [score]
        else:
            soln_pairs[key_ind_subset[id1]].append(key_ind_subset[id2])
            scores_key[key_ind_subset[id1]].append(score)  
filtered_pairs = {}
for k in soln_pairs.keys():
    filtered_pairs[k] = soln_pairs[k][0]

lst = []
new_pairs = {}
for k in filtered_pairs.keys():
    id2 = filtered_pairs[k]
    if id2 in filtered_pairs.keys():
        if 1.0 > scores_key[id2][0] > scores_key[k][0]:
            new_pairs[id2] = filtered_pairs[id2]
            lst.append(id2)
        else:
            new_pairs[k] = id2
    else:
        new_pairs[k] = id2

test_df = pd.DataFrame(new_pairs.items())
test_df.columns = ['id1','id2']
df_name = f"test_filtered_results.csv"
test_df.to_csv(df_name,index=False)
len(new_pairs), len(soln_pairs)

(5701, 6451)

In [43]:

preds = pd.read_csv('test_filtered_results.csv')

pairs = []
ids = []
for i,(k,v) in preds.iterrows():
    pairs.append((test_sentences[k], test_sentences[v]))
    ids.append((k,v))

# model_save_path = 'cross-encoder/stsb-roberta-large'
# cross_enc = CrossEncoder(model_save_path)
predictions_confs = cross_enc.predict(pairs)

Batches:   0%|          | 0/179 [00:00<?, ?it/s]

In [44]:
!mkdir /content/new_preds/

for th in [0.57, 0.58, 0.59, 0.6, 0.61, 0.62,0.63,0.64, 0.65, 0.66, 0.67, 0.68, 0.7]:
    #new method
    soln_pairs = {}
    for i,p in enumerate(predictions_confs):
        if p > th and not ids[i][1] in soln_pairs.values():
            soln_pairs[ids[i][0]] = ids[i][1]

    #moved filtering from intermediate results to this part based on cross encoder prediction results
    new_pairs = {}
    for k in soln_pairs.keys():
        id2 = soln_pairs[k]
        if id2 in soln_pairs.keys() and predictions_confs[[x[0] for x in ids].index(id2)] > predictions_confs[[x[0] for x in ids].index(k)]:
            new_pairs[id2] = soln_pairs[id2]
        else:
            new_pairs[k] = id2

    print(f"percentage retained for {th} is before filtering: ", len(soln_pairs)/len(ids))
    print(f"percentage retained for {th} is after filtering: ", len(new_pairs)/len(ids))

    print(f"length of final prediction pairs is: ", len(new_pairs))
    val_df = pd.DataFrame(new_pairs.items())
    val_df.columns = ['id1','id2']
    df_name = f"new_preds/test_cross_enc_result_{th}.csv"
    val_df.to_csv(df_name,index=False)

percentage retained for 0.57 is before filtering:  0.7809156288370461
percentage retained for 0.57 is after filtering:  0.7744255393790563
length of final prediction pairs is:  4415
percentage retained for 0.58 is before filtering:  0.7751271706718119
percentage retained for 0.58 is after filtering:  0.768812489037011
length of final prediction pairs is:  4383
percentage retained for 0.59 is before filtering:  0.768812489037011
percentage retained for 0.59 is after filtering:  0.7626732152253991
length of final prediction pairs is:  4348
percentage retained for 0.6 is before filtering:  0.7621469917558323
percentage retained for 0.6 is after filtering:  0.7563585335905981
length of final prediction pairs is:  4312
percentage retained for 0.61 is before filtering:  0.7579372039992983
percentage retained for 0.61 is after filtering:  0.7521487458340642
length of final prediction pairs is:  4288
percentage retained for 0.62 is before filtering:  0.750570075425364
percentage retained for 0

In [45]:
#comparing the length of val predictions and their results, I concluded best threshold would be 0.66

#now do post processing
sols = pd.read_csv('/content/new_preds/test_cross_enc_result_0.66.csv')
new_pairs = {}
duplicate = 0
for x, (id1,id2) in sols.iterrows():
    if test_sentences[id1] != test_sentences[id2]:
        duplicate += 1
        new_pairs[id1] = id2

print(f"percentage retained for is after filtering: ", len(new_pairs)/len(sols))
val_df = pd.DataFrame(new_pairs.items())
val_df.columns = ['id1','id2']
df_name = f"new_preds/test_cross_enc_result_final.csv"
val_df.to_csv(df_name,index=False)
print(f"Length {duplicate}")

percentage retained for is after filtering:  0.8220151998038735
Length 3353


In [46]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [47]:
!cp -r '/content/output/training_paraphrase_finetuning_training-paraphrase-multilingual-mpnet-base-v2-2022-07-01_03-28-24' '/content/gdrive/MyDrive/Paraphrase/fine_tuned_paraphrase-multilingual-mpnet-base-v2-final-model'